In [ ]:
def histo_hestetics(h):
    h.SetStats(0)
    h.GetYaxis().SetTitleSize(0.05)
    h.GetXaxis().SetTitleSize(0.05)


In [ ]:
def canvas_hestetics_layout(c, n=3, n1=0):
    c.SetLeftMargin(0.02)
    c.SetBottomMargin(0.02)
    c.SetRightMargin(0.02)
    c.SetTopMargin(0.02)
    c.SetTicky()
    c.SetTickx()
    c.Divide(n, n1)
    for i in range(1, n+1):
        c.cd(i).SetBottomMargin(0.1)
        c.cd(i).SetLeftMargin(0.1)
        c.cd(i).SetRightMargin(0.13)


def canvas_hestetics_single(c):
    c.SetTicky()
    c.SetTickx()


In [ ]:
from ROOT import TFile, TLatex, gPad
from plotting import draw_nice_canvas


def draw(run="505285"):
    names = ["hHits",
             "hSlotPartMask",
             "hIndexEOIsNoise",
             "hIndexEOInTimeWin",
             "hIndexEOHitRate",
             "TOFRawsMulti",
             "TOFRawHitMap",
             "TOFRawsTime",
             "OrbitDDL",
             "mean_of_hits"]
    h = {}
    for i in names:
        f = TFile(f"Run{run}/{i}.root", "READ")
        h[i] = f.Get("ccdb_object")
        h[i].SetDirectory(0)

    label = TLatex()
    label.SetTextFont(42)
    label.SetNDC()
    label.SetTextSize(0.06)
    label.SetTextAlign(22)
    label.SetTextAngle(0)

    cnoise = draw_nice_canvas("cnoise", 1600, 1250)
    canvas_hestetics_layout(cnoise, 0, 2)
    cnoise.cd(1).SetLogy()
    h["hIndexEOHitRate"].Draw("HIST")
    label.DrawLatex(0.25, 0.8, f"Run {run}")
    cnoise.cd(2).SetLogy()
    h["hIndexEOInTimeWin"].Draw("HIST")
    h["hIndexEOInTimeWin"].SetTitle("All channels, noisy in red")
    h["hIndexEOIsNoise"].SetLineColor(2)
    h["hIndexEOIsNoise"].Draw("HIST SAME")
    cnoise.SaveAs(f"Run{run}/noise_layout.png")

    craw = draw_nice_canvas("craw", 1800, 550)
    canvas_hestetics_layout(craw, 2)
    craw.cd(1).SetLogy()
    h["hHits"].SetLineWidth(2)
    h["hHits"].Draw()
    label. DrawLatex(0.25, 0.8, "Run {run}")
    craw.cd(2)
    h["hSlotPartMask"].Draw("colz")
    craw.cd(3)
    craw.SaveAs(f"Run{run}/raw_layout.png")

    cdig = draw_nice_canvas("cdig", 1800, 850)
    canvas_hestetics_layout(cdig, 2, 2)
    cdig.cd(1).SetLogz()
    h["TOFRawHitMap"].Draw("colz")
    label. DrawLatex(0.25, 0.8, f"Run {run}")
    cdig.cd(2)
    h["TOFRawsMulti"].Draw()
    cdig.cd(3)
    h["TOFRawsTime"].Draw()
    cdig.cd(4)
    h["OrbitDDL"].Draw("colz")
    cdig.SaveAs(f"Run{run}/digits_layout.png")

    ctrend = draw_nice_canvas("ctrend", 1800, 800)
    h["mean_of_hits"].Draw()
    ctrend.SaveAs("trending.png")

    for i in h:
        csingle = draw_nice_canvas(i, logz=True)
        h[i].Draw("colz HIST")
        label. DrawLatex(0.27, 0.8, f"Run {run}")
        csingle[i].SaveAs(f"Run{run}/{i}.png")
        csingle[i].SaveAs(f"Run{run}/{i}.root")
